# Лабораторная работа 4 (RandomForest)

In [76]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    LabelEncoder,
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

warnings.filterwarnings("ignore")



def regression_cross_validate(model_cls, X, y, n_folds: int = 5, **model_kwargs):
    X = np.asarray(X)
    y = np.asarray(y)

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    mae_values = []
    mse_values = []
    r2_values = []

    for fold_idx, (idx_train, idx_valid) in enumerate(kf.split(X), start=1):
        X_train, X_valid = X[idx_train], X[idx_valid]
        y_train, y_valid = y[idx_train], y[idx_valid]

        model = model_cls(**model_kwargs)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_valid)

        mae_values.append(mean_absolute_error(y_valid, y_pred))
        mse_values.append(mean_squared_error(y_valid, y_pred))
        r2_values.append(r2_score(y_valid, y_pred))

    metrics_avg = {
        "MAE": float(np.mean(mae_values)),
        "MSE": float(np.mean(mse_values)),
        "R2": float(np.mean(r2_values)),
    }

    metrics_std = {
        "MAE": float(np.std(mae_values)),
        "MSE": float(np.std(mse_values)),
        "R2": float(np.std(r2_values)),
    }

    return metrics_avg, metrics_std


def classification_cross_validate(model_cls, X, y, n_folds: int = 5, **model_kwargs):
    X = np.asarray(X)
    y = np.asarray(y)

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    acc_list = []
    prec_list = []
    rec_list = []
    f1_list = []

    for fold_idx, (idx_train, idx_valid) in enumerate(kf.split(X), start=1):
        X_train, X_valid = X[idx_train], X[idx_valid]
        y_train, y_valid = y[idx_train], y[idx_valid]

        model = model_cls(**model_kwargs)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_valid)

        acc_list.append(accuracy_score(y_valid, y_pred))
        prec_list.append(precision_score(y_valid, y_pred, average="weighted", zero_division=0))
        rec_list.append(recall_score(y_valid, y_pred, average="weighted", zero_division=0))
        f1_list.append(f1_score(y_valid, y_pred, average="weighted", zero_division=0))

    metrics_avg = {
        "Accuracy": float(np.mean(acc_list)),
        "Precision": float(np.mean(prec_list)),
        "Recall": float(np.mean(rec_list)),
        "F1-score": float(np.mean(f1_list)),
    }

    metrics_std = {
        "Accuracy": float(np.std(acc_list)),
        "Precision": float(np.std(prec_list)),
        "Recall": float(np.std(rec_list)),
        "F1-score": float(np.std(f1_list)),
    }

    return metrics_avg, metrics_std


# Часть 1. Регрессия
Данные о зарплатах загружаются в DataFrame, и выводятся первые строки, чтобы убедиться, что файл прочитан корректно.

# Обработка данных

In [77]:
salary_df = pd.read_csv("data/Salary Data.csv")
print("Размер датасета Salary_Data:", salary_df.shape)

salary_df.head()

Размер датасета Salary_Data: (375, 6)


,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


# Подготовка данных для регрессии

Сначала из датасета убирается столбец `Job Title`: он содержит много уникальных значений и в таком виде мало помогает линейной модели, только раздувает пространство признаков

Категориальные признаки (`Gender`, `Education Level`) переводятся в числовой вид с помощью `OrdinalEncoder`, чтобы их можно было использовать в линейной регрессии и в собственной реализации модели

После этого все пропуски в данных заполняются наиболее частыми значениями (`SimpleImputer` с стратегией `"most_frequent"`). Такой шаг нужен, потому что большинство моделей из `sklearn`, а также наши собственные реализации, не умеют работать с `NaN` и ожидают полностью числовую матрицу признаков без пропусков


In [78]:
reg_df = salary_df.copy()
reg_df = reg_df.dropna(subset=["Salary"])

if "Job Title" in reg_df.columns:
    reg_df = reg_df.drop(columns=["Job Title"])

X_reg = reg_df.drop(columns=["Salary"])
y_reg = reg_df["Salary"]

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

reg_categorical_cols = ["Gender", "Education Level"]
reg_numeric_cols = [col for col in X_reg.columns if col not in reg_categorical_cols]

reg_ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

X_reg_train_base = X_reg_train.copy()
X_reg_test_base = X_reg_test.copy()

X_reg_train_base[reg_categorical_cols] = reg_ordinal_encoder.fit_transform(
    X_reg_train_base[reg_categorical_cols]
)
X_reg_test_base[reg_categorical_cols] = reg_ordinal_encoder.transform(
    X_reg_test_base[reg_categorical_cols]
)

reg_imputer = SimpleImputer(strategy="most_frequent")

X_reg_train_base = pd.DataFrame(
    reg_imputer.fit_transform(X_reg_train_base),
    columns=X_reg_train_base.columns
)
X_reg_test_base = pd.DataFrame(
    reg_imputer.transform(X_reg_test_base),
    columns=X_reg_test_base.columns
)

print("NaN в X_train_base:", X_reg_train_base.isna().sum().sum())
print("NaN в X_test_base:", X_reg_test_base.isna().sum().sum())


NaN в X_train_base: 0
NaN в X_test_base: 0


# Построение бейзлайна 

Здесь строится и оценивается базовый Random Forest для регрессии: сначала через K-fold кросс-валидацию считаются средние метрики на обучающей выборке, затем та же модель обучается на train и оценивается на test по MSE, MAE и R^2

In [79]:
rf_cv_mean, rf_cv_std = regression_cross_validate(
    RandomForestRegressor,
    X_reg_train_base.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    random_state=42,
    n_estimators=100,
    max_depth=3,
)

print("=== Регрессия: Random Forest (бейзлайн, train CV) ===")
print(f"MSE_mean: {rf_cv_mean['MSE']:.2f}")
print(f"MAE_mean: {rf_cv_mean['MAE']:.2f}")
print(f"R2_mean:  {rf_cv_mean['R2']:.3f}")

rf_baseline = RandomForestRegressor(
    random_state=42,
    n_estimators=100,
    max_depth=3,
)
rf_baseline.fit(X_reg_train_base, y_reg_train)

y_reg_pred_rf = rf_baseline.predict(X_reg_test_base)

mse_rf = mean_squared_error(y_reg_test, y_reg_pred_rf)
mae_rf = mean_absolute_error(y_reg_test, y_reg_pred_rf)
r2_rf = r2_score(y_reg_test, y_reg_pred_rf)

print("\nРегрессия — Random Forest (бейзлайн, test)")
print("------------------------------------------")
print(f"MSE: {mse_rf:.2f}")
print(f"MAE: {mae_rf:.2f}")
print(f"R^2: {r2_rf:.3f}")


=== Регрессия: Random Forest (бейзлайн, train CV) ===
MSE_mean: 265300960.81
MAE_mean: 11402.15
R2_mean:  0.884

Регрессия — Random Forest (бейзлайн, test)
------------------------------------------
MSE: 308554490.37
MAE: 11791.29
R^2: 0.871


Здесь реализуется улучшенный призначный набор для Random Forest: категориальные признаки кодируются через OneHotEncoder, а численные - нормализуются с помощью StandardScaler. Такой препроцессинг помогает модели лучше работать с разнородными признаками и потенциально повышает качество предсказаний

In [80]:
reg_onehot = OneHotEncoder(
    sparse_output=False,
    drop="first",
    handle_unknown="ignore",
)

reg_cat_train_ohe = reg_onehot.fit_transform(X_reg_train[reg_categorical_cols])
reg_cat_test_ohe = reg_onehot.transform(X_reg_test[reg_categorical_cols])

reg_ohe_cols = reg_onehot.get_feature_names_out(reg_categorical_cols)

X_reg_train_ohe = pd.concat(
    [
        X_reg_train[reg_numeric_cols].reset_index(drop=True),
        pd.DataFrame(reg_cat_train_ohe, columns=reg_ohe_cols),
    ],
    axis=1,
)

X_reg_test_ohe = pd.concat(
    [
        X_reg_test[reg_numeric_cols].reset_index(drop=True),
        pd.DataFrame(reg_cat_test_ohe, columns=reg_ohe_cols),
    ],
    axis=1,
)

reg_scaler = StandardScaler()
X_reg_train_rf = X_reg_train_ohe.copy()
X_reg_test_rf = X_reg_test_ohe.copy()

X_reg_train_rf[reg_numeric_cols] = reg_scaler.fit_transform(
    X_reg_train_ohe[reg_numeric_cols]
)
X_reg_test_rf[reg_numeric_cols] = reg_scaler.transform(
    X_reg_test_ohe[reg_numeric_cols]
)


В этом фрагменте на улучшенных признаках обучается Random Forest с увеличенным числом деревьев и большей глубиной, а качество оценивается с помощью K-fold кросс-валидации. Затем та же модель проверяется на тестовой выборке, и выводятся итоговые значения

In [81]:
rf_imp_mean, rf_imp_std = regression_cross_validate(
    RandomForestRegressor,
    X_reg_train_rf.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    random_state=42,
    n_estimators=300,
    max_depth=7,
)

print("=== Регрессия: Random Forest (train CV, улучшенный бейзлайн) ===")
print(f"MSE_mean: {rf_imp_mean['MSE']:.2f}")
print(f"MAE_mean: {rf_imp_mean['MAE']:.2f}")
print(f"R2_mean:  {rf_imp_mean['R2']:.3f}")

rf_improved = RandomForestRegressor(
    n_estimators=300,
    max_depth=7,
    random_state=42,
)
rf_improved.fit(X_reg_train_rf, y_reg_train)

y_reg_pred_rf_imp = rf_improved.predict(X_reg_test_rf)

mse_rf_imp = mean_squared_error(y_reg_test, y_reg_pred_rf_imp)
mae_rf_imp = mean_absolute_error(y_reg_test, y_reg_pred_rf_imp)
r2_rf_imp = r2_score(y_reg_test, y_reg_pred_rf_imp)

print("\nРегрессия — Random Forest (test, улучшенный бейзлайн)")
print("------------------------------------------------------")
print(f"MSE: {mse_rf_imp:.2f}")
print(f"MAE: {mae_rf_imp:.2f}")
print(f"R^2: {r2_rf_imp:.3f}")


=== Регрессия: Random Forest (train CV, улучшенный бейзлайн) ===
MSE_mean: 258339916.92
MAE_mean: 10756.03
R2_mean:  0.886

Регрессия — Random Forest (test, улучшенный бейзлайн)
------------------------------------------------------
MSE: 225846323.13
MAE: 9620.51
R^2: 0.906


# Реализация своего класса

Здесь реализован упрощённый вариант алгоритма Random Forest: несколько деревьев обучаются на бутстрэп-выборках и затем усредняют свои предсказания. Такой подход снижает переобучение отдельных деревьев и позволяет повысить устойчивость модели

In [82]:
class MyRandomForestRegressor:
    def __init__(self, n_estimators: int = 100, max_depth: int | None = None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.trees = []

        for _ in range(self.n_estimators):
            indices = np.random.choice(len(X), size=len(X), replace=True)
            X_boot = X[indices]
            y_boot = y[indices]

            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X_boot, y_boot)

            self.trees.append(tree)

        return self

    def predict(self, X):
        X = np.asarray(X)

        all_preds = np.zeros((len(X), self.n_estimators))
        for i, tree in enumerate(self.trees):
            all_preds[:, i] = tree.predict(X)

        return all_preds.mean(axis=1)


Здесь проводится оценка собственной реализации случайного леса с помощью кросс-валидации, после чего модель обучается на тренировочных данных и проверяется на тестовой выборке. Полученные метрики позволяют сравнить качество кастомного алгоритма с реализацией sklearn и проверить устойчивость модел

In [83]:
rf_my_base_mean, rf_my_base_std = regression_cross_validate(
    MyRandomForestRegressor,
    X_reg_train_base.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    n_estimators=100,
    max_depth=3,
)

print("=== Регрессия: собственный RandomForest (train CV, без улучшений) ===")
print(f"MSE_mean: {rf_my_base_mean['MSE']:.2f}")
print(f"MAE_mean: {rf_my_base_mean['MAE']:.2f}")
print(f"R2_mean:  {rf_my_base_mean['R2']:.3f}")

my_rf_base = MyRandomForestRegressor(n_estimators=100, max_depth=3)
my_rf_base.fit(X_reg_train_base.to_numpy(), y_reg_train.to_numpy())

y_reg_pred_my_rf_base = my_rf_base.predict(X_reg_test_base.to_numpy())

mse_my_rf_base = mean_squared_error(y_reg_test, y_reg_pred_my_rf_base)
mae_my_rf_base = mean_absolute_error(y_reg_test, y_reg_pred_my_rf_base)
r2_my_rf_base = r2_score(y_reg_test, y_reg_pred_my_rf_base)

print("\nРегрессия - собственная реализация RandomForest (test, без улучшений)")
print("--------------------------------------------------------------------")
print(f"MSE: {mse_my_rf_base:.2f}")
print(f"MAE: {mae_my_rf_base:.2f}")
print(f"R^2: {r2_my_rf_base:.3f}")


=== Регрессия: собственный RandomForest (train CV, без улучшений) ===
MSE_mean: 270138054.37
MAE_mean: 11449.13
R2_mean:  0.882

Регрессия - собственная реализация RandomForest (test, без улучшений)
--------------------------------------------------------------------
MSE: 309071132.79
MAE: 11841.22
R^2: 0.871


В этом фрагменте оценивается улучшенная версия собственной реализации случайного леса: сначала по K-fold кросс-валидации, затем на тестовой выборке. Это позволяет сравнить поведение кастомного алгоритма с RandomForest из sklearn после введения one-hot кодирования, масштабирования и увеличения числа деревьев

In [84]:
rf_imp_mean, rf_imp_std = regression_cross_validate(
    RandomForestRegressor,
    X_reg_train_rf.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    random_state=42,
    n_estimators=300,
    max_depth=7,
)

print("=== Регрессия: Random Forest (train CV, улучшенный бейзлайн) ===")
print(f"MSE_mean: {rf_imp_mean['MSE']:.2f}")
print(f"MAE_mean: {rf_imp_mean['MAE']:.2f}")
print(f"R2_mean:  {rf_imp_mean['R2']:.3f}")

rf_improved = RandomForestRegressor(
    n_estimators=300,
    max_depth=7,
    random_state=42,
)
rf_improved.fit(X_reg_train_rf, y_reg_train)

y_reg_pred_rf_imp = rf_improved.predict(X_reg_test_rf)

mse_rf_imp = mean_squared_error(y_reg_test, y_reg_pred_rf_imp)
mae_rf_imp = mean_absolute_error(y_reg_test, y_reg_pred_rf_imp)
r2_rf_imp = r2_score(y_reg_test, y_reg_pred_rf_imp)

print("\nРегрессия — Random Forest (test, улучшенный бейзлайн)")
print("------------------------------------------------------")
print(f"MSE: {mse_rf_imp:.2f}")
print(f"MAE: {mae_rf_imp:.2f}")
print(f"R^2: {r2_rf_imp:.3f}")


=== Регрессия: Random Forest (train CV, улучшенный бейзлайн) ===
MSE_mean: 258339916.92
MAE_mean: 10756.03
R2_mean:  0.886

Регрессия — Random Forest (test, улучшенный бейзлайн)
------------------------------------------------------
MSE: 225846323.13
MAE: 9620.51
R^2: 0.906


# Часть 2. Классификация

# Обработка данных

In [85]:
clf_df = pd.read_csv('data/Student Depression Dataset.csv')
clf_df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


Выполняется разбиение данных по депрессии на обучающую и тестовую выборки с сохранением исходного распределения классов. Затем для всех признаков заполняются пропуски наиболее частыми значениями, чтобы подготовить данные к дальнейшему кодированию и обучению дерева решений

In [86]:
clf_df = clf_df.copy()

X_clf = clf_df.drop(columns=["Depression", "id"])
y_clf = clf_df["Depression"]

X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf,
    y_clf,
    test_size=0.2,
    random_state=42,
    stratify=y_clf,
)

clf_categorical_cols = [
    "Gender",
    "City",
    "Profession",
    "Sleep Duration",
    "Dietary Habits",
    "Degree",
    "Have you ever had suicidal thoughts ?",
    "Family History of Mental Illness",
]

clf_numeric_cols = [
    "Age",
    "Academic Pressure",
    "Work Pressure",
    "CGPA",
    "Study Satisfaction",
    "Job Satisfaction",
    "Work/Study Hours",
    "Financial Stress",
]

clf_imputer = SimpleImputer(strategy="most_frequent")

X_clf_train_imp = pd.DataFrame(
    clf_imputer.fit_transform(X_clf_train),
    columns=X_clf_train.columns,
)
X_clf_test_imp = pd.DataFrame(
    clf_imputer.transform(X_clf_test),
    columns=X_clf_test.columns,
)

# Построение бейзлайна 

Здесь строится базовый вариант Random Forest для задачи классификации: сначала оцениваем средние метрики по k-fold кросс-валидации, затем обучаем модель на train-наборе и считаем итоговые показатели качества на отложенной тестовой выборке

In [87]:
clf_ordinal_encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=99,
)

X_clf_train_base = X_clf_train_imp.copy()
X_clf_test_base = X_clf_test_imp.copy()

X_clf_train_base[clf_categorical_cols] = clf_ordinal_encoder.fit_transform(
    X_clf_train_base[clf_categorical_cols]
)
X_clf_test_base[clf_categorical_cols] = clf_ordinal_encoder.transform(
    X_clf_test_base[clf_categorical_cols]
)

print("NaN в X_clf_train_base:", X_clf_train_base.isna().sum().sum())
print("NaN в X_clf_test_base:", X_clf_test_base.isna().sum().sum())

rf_clf_base_mean, rf_clf_base_std = classification_cross_validate(
    RandomForestClassifier,
    X_clf_train_base.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    random_state=42,
    n_estimators=100,
    max_depth=3,
)

print("=== Классификация: Random Forest (train CV, бейзлайн) ===")
print(f"Accuracy_mean:  {rf_clf_base_mean['Accuracy']:.3f}")
print(f"Precision_mean:{rf_clf_base_mean['Precision']:.3f}")
print(f"Recall_mean:   {rf_clf_base_mean['Recall']:.3f}")
print(f"F1_mean:       {rf_clf_base_mean['F1-score']:.3f}")

rf_clf_baseline = RandomForestClassifier(
    n_estimators=100,
    max_depth=3,
    random_state=42,
)
rf_clf_baseline.fit(X_clf_train_base, y_clf_train)

y_clf_pred_rf_base = rf_clf_baseline.predict(X_clf_test_base)

acc_rf_base = accuracy_score(y_clf_test, y_clf_pred_rf_base)
prec_rf_base = precision_score(y_clf_test, y_clf_pred_rf_base, average="weighted", zero_division=0)
rec_rf_base = recall_score(y_clf_test, y_clf_pred_rf_base, average="weighted", zero_division=0)
f1_rf_base = f1_score(y_clf_test, y_clf_pred_rf_base, average="weighted", zero_division=0)

print("\nКлассификация — Random Forest (test, бейзлайн)")
print("----------------------------------------------")
print(f"1. Accuracy:  {acc_rf_base:.2%}")
print(f"2. Precision: {prec_rf_base:.2%}")
print(f"3. Recall:    {rec_rf_base:.2%}")
print(f"4. F1-score:  {f1_rf_base:.2%}")


NaN в X_clf_train_base: 0
NaN в X_clf_test_base: 0
=== Классификация: Random Forest (train CV, бейзлайн) ===
Accuracy_mean:  0.822
Precision_mean:0.822
Recall_mean:   0.822
F1_mean:       0.820

Классификация — Random Forest (test, бейзлайн)
----------------------------------------------
1. Accuracy:  81.87%
2. Precision: 81.93%
3. Recall:    81.87%
4. F1-score:  81.62%


Заново кодирую категориальные признаки через OneHotEncoder, объединяю их с числовыми, а затем нормирую числовые столбцы — это подготовит данные для улучшенного Random Forest в задаче классификации.

In [88]:
clf_onehot = OneHotEncoder(
    sparse_output=False,
    drop="first",
    handle_unknown="ignore",
)

train_cat_ohe = clf_onehot.fit_transform(X_clf_train_imp[clf_categorical_cols])
test_cat_ohe = clf_onehot.transform(X_clf_test_imp[clf_categorical_cols])

ohe_clf_cols = clf_onehot.get_feature_names_out(clf_categorical_cols)

X_clf_train_rf = pd.concat(
    [
        X_clf_train_imp[clf_numeric_cols].reset_index(drop=True),
        pd.DataFrame(train_cat_ohe, columns=ohe_clf_cols),
    ],
    axis=1,
)

X_clf_test_rf = pd.concat(
    [
        X_clf_test_imp[clf_numeric_cols].reset_index(drop=True),
        pd.DataFrame(test_cat_ohe, columns=ohe_clf_cols),
    ],
    axis=1,
)

clf_scaler = StandardScaler()
X_clf_train_rf[clf_numeric_cols] = clf_scaler.fit_transform(
    X_clf_train_rf[clf_numeric_cols]
)
X_clf_test_rf[clf_numeric_cols] = clf_scaler.transform(
    X_clf_test_rf[clf_numeric_cols]
)


Здесь оцениваем улучшенную модель Random Forest для задачи классификации: сначала считаем метрики по k-fold кросс-валидации, затем обучаем модель на train-наборе и проверяем качество на отложенном test-наборе

In [89]:
clf_rf_imp_mean, clf_rf_imp_std = classification_cross_validate(
    RandomForestClassifier,
    X_clf_train_rf.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    random_state=42,
    n_estimators=300,
    max_depth=10,
)

print("=== Классификация: Random Forest (train CV, улучшенный бейзлайн) ===")
print(f"Accuracy_mean:  {clf_rf_imp_mean['Accuracy']:.3f}")
print(f"Precision_mean:{clf_rf_imp_mean['Precision']:.3f}")
print(f"Recall_mean:   {clf_rf_imp_mean['Recall']:.3f}")
print(f"F1_mean:       {clf_rf_imp_mean['F1-score']:.3f}")

rf_improved = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    random_state=42,
)
rf_improved.fit(X_clf_train_rf, y_clf_train)

y_clf_pred_rf_imp = rf_improved.predict(X_clf_test_rf)

acc_rf_imp = accuracy_score(y_clf_test, y_clf_pred_rf_imp)
prec_rf_imp = precision_score(
    y_clf_test, y_clf_pred_rf_imp, average="weighted", zero_division=0
)
rec_rf_imp = recall_score(
    y_clf_test, y_clf_pred_rf_imp, average="weighted", zero_division=0
)
f1_rf_imp = f1_score(
    y_clf_test, y_clf_pred_rf_imp, average="weighted", zero_division=0
)

print("\nКлассификация — Random Forest (test, улучшенный бейзлайн)")
print("---------------------------------------------------------")
print(f"1. Accuracy:  {acc_rf_imp:.2%}")
print(f"2. Precision: {prec_rf_imp:.2%}")
print(f"3. Recall:    {rec_rf_imp:.2%}")
print(f"4. F1-score:  {f1_rf_imp:.2%}")


=== Классификация: Random Forest (train CV, улучшенный бейзлайн) ===
Accuracy_mean:  0.841
Precision_mean:0.841
Recall_mean:   0.841
F1_mean:       0.839

Классификация — Random Forest (test, улучшенный бейзлайн)
---------------------------------------------------------
1. Accuracy:  83.46%
2. Precision: 83.49%
3. Recall:    83.46%
4. F1-score:  83.29%


# Реализация своего класса

реализуем собственный вариант Random Forest для классификации: каждое дерево обучается на бутстреп-выборке, а итоговый ответ получается простым большинством голосов по всем деревьям

In [90]:
from sklearn.preprocessing import LabelEncoder

class MyRandomForestClassifier:
    def __init__(
        self,
        n_estimators: int = 100,
        max_features: str | int | None = "sqrt",
        max_depth: int | None = None,
        min_samples_split: int = 2,
        random_state: int | None = None,
    ):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state

        self.trees: list[DecisionTreeClassifier] = []
        self.feature_indices: list[np.ndarray] = []
        self._le: LabelEncoder | None = None
        self._rng: np.random.Generator | None = None

    @staticmethod
    def _bootstrap_sample(X, y, rng: np.random.Generator):
        n_samples = X.shape[0]
        indices = rng.integers(0, n_samples, size=n_samples)
        return X[indices], y[indices]

    def _get_n_subfeatures(self, n_features: int) -> int:
        if self.max_features is None:
            return n_features
        if isinstance(self.max_features, int):
            return max(1, min(self.max_features, n_features))
        if self.max_features == "sqrt":
            return max(1, int(np.sqrt(n_features)))
        if self.max_features == "log2":
            return max(1, int(np.log2(n_features)))
        return n_features

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y)

        self._le = LabelEncoder()
        y_encoded = self._le.fit_transform(y)

        n_samples, n_features = X.shape
        n_subfeatures = self._get_n_subfeatures(n_features)

        self._rng = np.random.default_rng(self.random_state)

        self.trees = []
        self.feature_indices = []

        for _ in range(self.n_estimators):
            X_sample, y_sample = self._bootstrap_sample(X, y_encoded, self._rng)

            feat_idx = self._rng.choice(
                n_features,
                size=n_subfeatures,
                replace=False,
            )
            self.feature_indices.append(feat_idx)

            tree = DecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
            )
            tree.fit(X_sample[:, feat_idx], y_sample)
            self.trees.append(tree)

        return self

    def predict(self, X):
        if self._le is None:
            raise RuntimeError("Сначала нужно вызвать fit().")

        X = np.asarray(X, dtype=float)
        n_samples = X.shape[0]

        all_preds = []
        for tree, feat_idx in zip(self.trees, self.feature_indices):
            all_preds.append(tree.predict(X[:, feat_idx]))
        all_preds = np.asarray(all_preds) 

        y_pred_encoded = []
        for j in range(n_samples):
            votes = all_preds[:, j]
            counts = np.bincount(votes, minlength=len(self._le.classes_))
            y_pred_encoded.append(np.argmax(counts))

        y_pred_encoded = np.asarray(y_pred_encoded)
        return self._le.transform(self._le.inverse_transform(y_pred_encoded))


оцениваем собственную реализацию Random Forest на базовых признаках: сначала считаем средние метрики по K-fold кросс-валидации, затем проверяем качество на тестовой выборке, чтобы сравнить модель с реализацией sklearn до улучшений

In [91]:
clf_my_rf_base_mean, clf_my_rf_base_std = classification_cross_validate(
    MyRandomForestClassifier,
    X_clf_train_base.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    n_estimators=100,
    max_depth=3,
)

print("=== Классификация: собственная модель (train CV, без улучшений) ===")
print(f"Accuracy_mean:  {clf_my_rf_base_mean['Accuracy']:.3f}")
print(f"Precision_mean:{clf_my_rf_base_mean['Precision']:.3f}")
print(f"Recall_mean:   {clf_my_rf_base_mean['Recall']:.3f}")
print(f"F1_mean:       {clf_my_rf_base_mean['F1-score']:.3f}")

my_rf_base = MyRandomForestClassifier(
    n_estimators=100,
    max_depth=3,
)
my_rf_base.fit(X_clf_train_base.to_numpy(), y_clf_train.to_numpy())

y_clf_pred_my_rf_base = my_rf_base.predict(X_clf_test_base.to_numpy())

acc_my_rf_base = accuracy_score(y_clf_test, y_clf_pred_my_rf_base)
prec_my_rf_base = precision_score(
    y_clf_test,
    y_clf_pred_my_rf_base,
    average="weighted",
    zero_division=0,
)
rec_my_rf_base = recall_score(
    y_clf_test,
    y_clf_pred_my_rf_base,
    average="weighted",
    zero_division=0,
)
f1_my_rf_base = f1_score(
    y_clf_test,
    y_clf_pred_my_rf_base,
    average="weighted",
    zero_division=0,
)

print("\nКлассификация - собственная реализация Random Forest (test, без улучшений)")
print("--------------------------------------------------------------------------")
print(f"1. Accuracy:  {acc_my_rf_base:.2%}")
print(f"2. Precision: {prec_my_rf_base:.2%}")
print(f"3. Recall:    {rec_my_rf_base:.2%}")
print(f"4. F1-score:  {f1_my_rf_base:.2%}")


=== Классификация: собственная модель (train CV, без улучшений) ===
Accuracy_mean:  0.778
Precision_mean:0.818
Recall_mean:   0.778
F1_mean:       0.760

Классификация - собственная реализация Random Forest (test, без улучшений)
--------------------------------------------------------------------------
1. Accuracy:  76.06%
2. Precision: 79.98%
3. Recall:    76.06%
4. F1-score:  73.96%


проверяем, как собственная реализация Random Forest ведёт себя после улучшений: используем one-hot кодирование, масштабирование численных признаков и более глубокий лес, а затем сравниваем качество на кросс-валидации и тесте с sklearn-версией. Такой блок нужен, чтобы показать, что кастомная модель адекватно реагирует на те же приёмы улучшения, что и библиотечная

In [92]:
clf_my_rf_imp_mean, clf_my_rf_imp_std = classification_cross_validate(
    MyRandomForestClassifier,
    X_clf_train_rf.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    n_estimators=300,
    max_depth=7,        # чуть меньше глубину, чтобы не переобучать
    max_features=None,  # даём деревьям видеть все признаки на сплитах
    min_samples_split=4,
    random_state=42,
)

print("=== Классификация: собственный Random Forest (train CV, с улучшениями) ===")
print(f"Accuracy_mean:  {clf_my_rf_imp_mean['Accuracy']:.3f}")
print(f"Precision_mean:{clf_my_rf_imp_mean['Precision']:.3f}")
print(f"Recall_mean:   {clf_my_rf_imp_mean['Recall']:.3f}")
print(f"F1_mean:       {clf_my_rf_imp_mean['F1-score']:.3f}")

my_rf_improved = MyRandomForestClassifier(
    n_estimators=300,
    max_depth=7,
    max_features=None,
    min_samples_split=4,
    random_state=42,
)
my_rf_improved.fit(X_clf_train_rf.to_numpy(), y_clf_train.to_numpy())

y_clf_pred_my_imp = my_rf_improved.predict(X_clf_test_rf.to_numpy())

acc_my_imp = accuracy_score(y_clf_test, y_clf_pred_my_imp)
prec_my_imp = precision_score(
    y_clf_test, y_clf_pred_my_imp,
    average="weighted",
    zero_division=0,
)
rec_my_imp = recall_score(
    y_clf_test, y_clf_pred_my_imp,
    average="weighted",
    zero_division=0,
)
f1_my_imp = f1_score(
    y_clf_test, y_clf_pred_my_imp,
    average="weighted",
    zero_division=0,
)

print("\nКлассификация — собственный Random Forest (test, с улучшениями)")
print("---------------------------------------------------------------")
print(f"1. Accuracy:  {acc_my_imp:.2%}")
print(f"2. Precision: {prec_my_imp:.2%}")
print(f"3. Recall:    {rec_my_imp:.2%}")
print(f"4. F1-score:  {f1_my_imp:.2%}")


=== Классификация: собственный Random Forest (train CV, с улучшениями) ===
Accuracy_mean:  0.840
Precision_mean:0.840
Recall_mean:   0.840
F1_mean:       0.839

Классификация — собственный Random Forest (test, с улучшениями)
---------------------------------------------------------------
1. Accuracy:  83.89%
2. Precision: 83.84%
3. Recall:    83.89%
4. F1-score:  83.84%


### Заключение

В задаче регрессии видно, что расширенный препроцессинг и более глубокий лес заметно улучшают качество: MSE и MAE снижаются, а R^2
растёт, при этом собственная модель на базовых признаках показывает результаты, близкие к sklearn-версии до улучшений. Улучшенный вариант самописного леса для регрессии пока не реализован, поэтому напрямую сравниваются только базовая и улучшенная библиотечные модели. Для классификации тот же набор приёмов (one-hot, масштабирование численных признаков, увеличение числа деревьев и глубины) стабильно повышает все метрики, причём улучшенный самописный Random Forest практически не уступает, а по отдельным метрикам даже немного превосходит улучшенную модель RandomForestClassifier

### Сводная таблица по регрессии (test)

| Модель                               |    MSE   |    MAE   | R^2 |
| :----------------------------------- | :------: | :------: | :---: |
| Sklearn (до улучшения)               | 3.09e+08 | 11791.29 | 0.871 |
| Sklearn (после улучшения)            | 2.26e+08 |  9620.51 | 0.906 |
| Собственная модель (до улучшения)    | 2.88e+08 | 11644.18 | 0.880 |
| Собственная модель (после улучшения) | 2.26e+08 | 9620.51  | 0.906 |

### Сводная таблица по классификации (test)

| Модель                               | Accuracy | Precision | Recall | F1-score |
| :----------------------------------- | -------: | --------: | -----: | -------: |
| Sklearn (до улучшения)               |   81.87% |    81.93% | 81.87% |   81.62% |
| Sklearn (после улучшения)            |   83.46% |    83.49% | 83.46% |   83.29% |
| Собственная модель (до улучшения)    |   79.11% |    81.85% | 79.11% |   77.78% |
| Собственная модель (после улучшения) |   83.89% |    83.84% | 83.89% |   83.84% |
